In [ ]:
# pip install xarray netcdf4
# !pip install xarray netcdf4 requests
# !pip install tensorflow
# !pip install requests xarray pandas scikit-learn matplotlib streamlit
# !pip install streamlit

In [ ]:
# Project Plan: Quantum and AI for Improved Typhoon Prediction in Taiwan
# Project Overview
# This project aims to leverage quantum computing and AI to enhance the accuracy and 
# prediction window of typhoon models. 
# The goal is to provide early warnings, improve trajectory predictions, and assess
# the severity of storms more accurately, 
# giving authorities and citizens more time to prepare and respond.

# Problem Statement
# Current typhoon prediction models provide a 2-3 day heads-up and predict 1-2 days ahead after landfall. 
# The objective is to extend this prediction window to a week and improve 
# the accuracy of trajectory and severity forecasts.

# Objectives
# Develop a hybrid quantum-classical algorithm for typhoon prediction.
# Utilize quantum simulations to enhance trajectory and severity predictions.
# Apply AI techniques to analyze and interpret large-scale typhoon data.
# Provide actionable insights and early warnings for disaster preparedness.

In [ ]:
# Relevance to Climate and Environmental Simulations: The project focuses on enhancing typhoon prediction, 
# directly related to climate and environmental simulations.
# Application of Quantum Computing and AI: It leverages both quantum computing and AI to improve prediction accuracy 
# and extend the forecasting window.
# High-Performance Modeling: It addresses the need for high-performance modeling and simulation, 
# a core interest of NNL.
# Innovation and Research Focus: The project involves researching state-of-the-art techniques 
# and potentially contributing new findings, which aligns with NNL’s emphasis on pioneering research.

In [ ]:
# Methodology
# 1. Literature Review:

# Study existing typhoon prediction models and their limitations.
# Research quantum computing algorithms relevant to weather simulations.
# Investigate AI methods for large-scale data analysis and predictive modeling.
# 2. Data Collection:

# Source historical typhoon data and real-time weather data from reliable datasets (e.g., CWB, NASA, NOAA).
# Preprocess and clean the data for analysis.

In [ ]:
# # Inspect the columns of the merged_data DataFrame
# print(merged_data.columns)

In [1]:
import os
import requests

# Define the list of URLs for all datasets
ersst_urls = [
    "https://www.ncei.noaa.gov/pub/data/cmb/ersst/v4/netcdf/ersst.v4.202001.nc",
    "https://www.ncei.noaa.gov/pub/data/cmb/ersst/v4/netcdf/ersst.v4.202002.nc",
]

avhrr_urls = [
    "https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2/access/avhrr-only/202001/avhrr-only-v2.20200101.nc",
    "https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2/access/avhrr-only/202002/avhrr-only-v2.20200201.nc",
]

gst_url = "https://www.ncei.noaa.gov/data/noaa-global-surface-temperature/v5/access/timeseries/aravg.mon.ocean.90S.90N.v5.0.0.202212.asc"

ghcn_urls = [
    "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz",
    "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/2021.csv.gz",
]

# Function to download and save data
def download_data(url, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    filename = url.split('/')[-1]
    filepath = os.path.join(folder, filename)
    response = requests.get(url)
    with open(filepath, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

# Download datasets
download_data(gst_url, 'gst_data')
for url in ersst_urls:
    download_data(url, 'ersst_v4_data')
for url in avhrr_urls:
    download_data(url, 'avhrr_data')
for url in ghcn_urls:
    download_data(url, 'ghcn_data')

Downloaded aravg.mon.ocean.90S.90N.v5.0.0.202212.asc
Downloaded ersst.v4.202001.nc
Downloaded ersst.v4.202002.nc
Downloaded avhrr-only-v2.20200101.nc
Downloaded avhrr-only-v2.20200201.nc
Downloaded 2020.csv.gz
Downloaded 2021.csv.gz


In [2]:
import xarray as xr
import pandas as pd

# Function to load multiple NetCDF files into a single xarray Dataset
def load_datasets(folder, ext='.nc'):
    data_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(ext)]
    datasets = [xr.open_dataset(file) for file in data_files]
    combined_ds = xr.concat(datasets, dim='time')
    return combined_ds

# Load NetCDF datasets
ersst_data = load_datasets('ersst_v4_data')
avhrr_data = load_datasets('avhrr_data')

# Load ASCII datasets (for GST)
def load_ascii_data(file_path, delimiter=r"\s+"):
    data = pd.read_csv(file_path, delimiter=delimiter, header=None)
    return data

gst_data = load_ascii_data('gst_data/aravg.mon.ocean.90S.90N.v5.0.0.202212.asc')

# Load CSV.gz datasets (for GHCN)
def load_csv_data(folder):
    data_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv.gz')]
    data_list = []
    for file in data_files:
        data = pd.read_csv(file, compression='gzip', header=None)
        data_list.append(data)
    combined_data = pd.concat(data_list, axis=0)
    return combined_data

ghcn_data = load_csv_data('ghcn_data')

In [3]:
def normalize(data):
    return (data - data.mean()) / data.std()

def extract_features(data, var_name):
    df = data[var_name].to_dataframe().reset_index()
    df[var_name] = normalize(df[var_name])
    return df

ersst_sst = extract_features(ersst_data, 'sst')
ersst_ssta = extract_features(ersst_data, 'ssta')
avhrr_sst = extract_features(avhrr_data, 'sst')
avhrr_anom = extract_features(avhrr_data, 'anom')

# Ensure 'zlev' is present in the DataFrames
ersst_sst['zlev'] = 0
ersst_ssta['zlev'] = 0
if 'zlev' not in avhrr_sst.columns:
    avhrr_sst['zlev'] = 0
if 'zlev' not in avhrr_anom.columns:
    avhrr_anom['zlev'] = 0

In [4]:
from sklearn.neighbors import NearestNeighbors

# Function to resample AVHRR data to match ERSST resolution
def resample_avhrr_data(avhrr_df, ersst_df):
    avhrr_resampled = avhrr_df.groupby(['time', 'lat', 'lon']).mean().reset_index()
    return avhrr_resampled

avhrr_sst_resampled = resample_avhrr_data(avhrr_sst, ersst_sst)
avhrr_anom_resampled = resample_avhrr_data(avhrr_anom, ersst_sst)

# Prepare nearest neighbors model for lat/lon matching
def match_nearest_neighbors(df1, df2, on_columns):
    nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(df2[on_columns].dropna())
    distances, indices = nn.kneighbors(df1[on_columns].dropna())
    matched_df = df1.copy()
    matched_df.loc[df1[on_columns].dropna().index, on_columns] = df2.iloc[indices.flatten()][on_columns].values
    return matched_df

matched_avhrr_sst = match_nearest_neighbors(avhrr_sst_resampled, ersst_sst, ['lat', 'lon'])
matched_avhrr_anom = match_nearest_neighbors(avhrr_anom_resampled, ersst_sst, ['lat', 'lon'])

In [5]:
# Merge all datasets into a single DataFrame for modeling
merged_data = ersst_sst.merge(ersst_ssta, on=['time', 'lat', 'lon', 'lev', 'zlev'], suffixes=('_sst', '_ssta'), how='inner')
merged_data = merged_data.merge(matched_avhrr_sst, on=['time', 'lat', 'lon', 'zlev'], suffixes=('', '_avhrr_sst'), how='inner')
merged_data = merged_data.merge(matched_avhrr_anom, on=['time', 'lat', 'lon', 'zlev'], suffixes=('', '_avhrr_anom'), how='inner')

# Drop rows with NaN values
merged_data.dropna(inplace=True)

# Check the columns and a sample of the merged_data DataFrame
print("Merged DataFrame Columns:")
print(merged_data.columns)
print("First 5 rows of merged_data:")
print(merged_data.head())

# Define features and target for LSTM
feature_cols = ['sst', 'ssta', 'sst_avhrr_sst', 'anom']
target_col = 'sst'

X = merged_data[feature_cols].values
y = merged_data[target_col].values

# Check the shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Inspect first few rows of X and y to ensure data is not empty
print(f"First 5 rows of X:\n{X[:5]}")
print(f"First 5 rows of y:\n{y[:5]}")

Merged DataFrame Columns:
Index(['time', 'lev', 'lat', 'lon', 'sst', 'zlev', 'ssta', 'sst_avhrr_sst',
       'anom'],
      dtype='object')
First 5 rows of merged_data:
              time  lev   lat    lon       sst  zlev      ssta  sst_avhrr_sst  \
5692928 2020-01-01  0.0 -76.0  164.0 -1.273254     0 -0.330044      -1.310388   
5692929 2020-01-01  0.0 -76.0  164.0 -1.273254     0 -0.330044      -1.310388   
5692930 2020-01-01  0.0 -76.0  164.0 -1.273254     0 -0.330044      -1.310388   
5692931 2020-01-01  0.0 -76.0  164.0 -1.273254     0 -0.330044      -1.310388   
5692932 2020-01-01  0.0 -76.0  164.0 -1.273254     0 -0.330044      -1.310388   

             anom  
5692928 -0.838026  
5692929 -0.838026  
5692930 -0.814510  
5692931 -0.755719  
5692932 -0.673412  
Shape of X: (86654622, 4)
Shape of y: (86654622,)
First 5 rows of X:
[[-1.2732539  -0.33004436 -1.310388   -0.8380261 ]
 [-1.2732539  -0.33004436 -1.310388   -0.8380261 ]
 [-1.2732539  -0.33004436 -1.310388   -0.8145098 ]
 [

In [8]:
import numpy as np  # Ensure numpy is imported

# Prepare LSTM data
def prepare_lstm_data(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i+time_steps])
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

X_lstm, y_lstm = prepare_lstm_data(X, y)

# Check the shapes of X_lstm and y_lstm
print(f"Shape of X_lstm: {X_lstm.shape}")
print(f"Shape of y_lstm: {y_lstm.shape}")

# Split data into training and testing sets
from sklearn.model_selection import train_test_split  # Ensure train_test_split is imported

if X_lstm.shape[0] > 0:  # Ensure there is data to split
    X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)
    print(f"Shape of X_train: {X_train.shape}")
    print(f"Shape of X_test: {X_test.shape}")
    print(f"Shape of y_train: {y_train.shape}")
    print(f"Shape of y_test: {y_test.shape}")
else:
    print("No data available for splitting.")

Shape of X_lstm: (86654612, 10, 4)
Shape of y_lstm: (86654612,)
Shape of X_train: (69323689, 10, 4)
Shape of X_test: (17330923, 10, 4)
Shape of y_train: (69323689,)
Shape of y_test: (17330923,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Ensure X_train is defined and has the correct shape before running this block
if 'X_train' in globals() and X_train.shape[0] > 0:
    # Define LSTM model
    model = Sequential([
        tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2])),
        LSTM(50, return_sequences=True),
        LSTM(50),
        Dense(1, activation='linear')
    ])

    # Compile model
    model.compile(optimizer='adam', loss='mse')

    # Train model
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    from sklearn.metrics import mean_absolute_error, mean_squared_error

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"Mean Absolute Error: {mae}")
    print(f"Root Mean Squared Error: {rmse}")

    # Save the model in the native Keras format
    model.save('my_model.keras')

    # Plot predictions vs actual values
    import matplotlib.pyplot as plt

    plt.figure(figsize=(14, 7))
    plt.plot(y_test, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.legend()
    plt.show()
else:
    print("Training data is not defined or has incorrect shape.")

Epoch 1/20
1733093/1733093 ━━━━━━━━━━━━━━━━━━━━ 8382s 5ms/step - loss: 6.1081e-05 - val_loss: 1.1038e-06
Epoch 2/20
1733093/1733093 ━━━━━━━━━━━━━━━━━━━━ 8299s 5ms/step - loss: 1.3777e-06 - val_loss: 1.0243e-06
Epoch 3/20
1733093/1733093 ━━━━━━━━━━━━━━━━━━━━ 8151s 5ms/step - loss: 1.3532e-06 - val_loss: 1.0535e-06
Epoch 4/20
1733093/1733093 ━━━━━━━━━━━━━━━━━━━━ 8425s 5ms/step - loss: 1.1017e-06 - val_loss: 1.0277e-06
Epoch 5/20
 555167/1733093 ━━━━━━━━━━━━━━━━━━━━ 1:46:17 5ms/step - loss: 1.3414e-06

In [ ]:
def predict_real_time(new_data):
    # Process new_data to match the format of X_train
    # new_data should be a DataFrame or array similar to merged_data
    X_new = new_data[feature_cols].values.reshape((1, new_data.shape[0], len(feature_cols)))
    prediction = model.predict(X_new)
    return prediction

# Example usage
# new_data = pd.DataFrame([...])  # New data in the same format as merged_data
# print(predict_real_time(new_data))

#### import streamlit as st

st.title('Typhoon Prediction Dashboard')

# Ensure y_test and y_pred are defined before plotting
if 'y_test' in globals() and 'y_pred' in globals():
    st.line_chart({'Actual': y_test, 'Predicted': y_pred})
else:
    st.write("y_test and y_pred are not defined.")

In [ ]:
# import streamlit as st

# st.title('Typhoon Prediction Dashboard')

# # Ensure y_test and y_pred are defined before plotting
# if 'y_test' in globals() and 'y_pred' in globals():
#     st.line_chart({'Actual': y_test, 'Predicted': y_pred})
# else:
#     st.write("y_test and y_pred are not defined.")

In [ ]:
# import streamlit as st

# st.title('Typhoon Prediction Dashboard')

# # Ensure y_test and y_pred are defined before plotting
# if 'y_test' in globals() and 'y_pred' in globals():
#     st.line_chart({'Actual': y_test[:1000], 'Predicted': y_pred[:1000]})  # Plot only a subset for better performance
# else:
#     st.write("y_test and y_pred are not defined.")

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

# # Adjusting the LSTM units and batch size
# model = Sequential()
# model.add(tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2])))
# model.add(LSTM(100, return_sequences=True))  # Adjusted LSTM units
# model.add(LSTM(100))  # Adjusted LSTM units
# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mse')

# # Train the model with different batch size
# model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

In [ ]:
# def predict_real_time(new_data):
#     # Process new_data to match the format of X_train
#     # new_data should be a DataFrame or array similar to merged_data
#     X_new = new_data[feature_cols].values.reshape((1, new_data.shape[0], len(feature_cols)))
#     prediction = model.predict(X_new)
#     return prediction

# # Example usage
# # new_data = pd.DataFrame([...])  # New data in the same format as merged_data
# # print(predict_real_time(new_data))

In [ ]:
# import streamlit as st

# st.title('Typhoon Prediction Dashboard')

# # Ensure y_test and y_pred are defined before plotting
# if 'y_test' in globals() and 'y_pred' in globals():
#     st.line_chart({'Actual': y_test, 'Predicted': y_pred})
# else:
#     st.write("y_test and y_pred are not defined.")

In [ ]:
print('stop here wed 24 2024')

In [ ]:
# Let's visualize the sea surface temperature anomalies over time to identify patterns that 
# could be indicative of typhoon formation.

In [ ]:
# import matplotlib.pyplot as plt

# # Plot sea surface temperature anomalies
# ssta_data.mean(dim='time').plot(cmap='coolwarm')
# plt.title('Average Sea Surface Temperature Anomalies')
# plt.show()

In [ ]:
# Building a Predictive Model
# To build a predictive model for typhoon prediction, we'll use historical typhoon data 
# and combine it with the preprocessed climate data. For this example, 
# we'll use a simple machine learning model (e.g., Random Forest) to predict 
# the occurrence of a typhoon based on sea surface temperature anomalies.

# Load Historical Typhoon Data:

# Obtain historical typhoon data from reliable sources such as NOAA or other meteorological databases.
# Preprocess the typhoon data to align with the climate data (time, location, etc.).


In [ ]:
# Next Steps:
# Enhance the Model:

# Experiment with different machine learning algorithms (e.g., LSTM, Gradient Boosting).
# Implement feature engineering to extract more relevant features from the datasets.
# Integrate Quantum Computing:

# Use quantum algorithms to optimize the predictive model.
# Leverage quantum machine learning techniques to enhance the model's performance.
# Deploy the Model:

# Create a real-time prediction system to provide early warnings for typhoons.
# Develop a visualization dashboard to monitor predictions and alert relevant authorities.
# By following these steps, you can develop a robust model for improved typhoon prediction using 
# quantum computing and AI, potentially providing more accurate and timely warnings to 
# enhance disaster preparedness and response in Taiwan.

In [ ]:
# Enhance the Model:
# Experiment with Different Machine Learning Algorithms:

# LSTM (Long Short-Term Memory): Use LSTM networks to capture temporal dependencies in the climate data, 
# which can improve the prediction of typhoon trajectories and intensities.

In [ ]:
# Implement Feature Engineering:

# Extract Additional Features:
# Sea surface temperature gradients
# Atmospheric pressure differences
# Historical typhoon paths

# Integrate Quantum Computing:
# Use Quantum Algorithms to Optimize the Predictive Model:

# Utilize Qiskit for quantum optimization tasks.
# Quantum Support Vector Machine (QSVM):

In [ ]:
# combined_data['sst_gradient'] = combined_data['sst'].differentiate('lat') + combined_data['sst'].differentiate('lon')
# combined_data['pressure_diff'] = combined_data['pressure'].differentiate('time')



# from qiskit import Aer
# from qiskit_machine_learning.algorithms import QSVM
# from qiskit_machine_learning.kernels import QuantumKernel
# from qiskit.circuit.library import ZZFeatureMap

# # Define feature map and quantum kernel
# feature_map = ZZFeatureMap(feature_dimension=3, reps=2, entanglement='linear')
# quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=Aer.get_backend('qasm_simulator'))

# # Define and train QSVM
# qsvm = QSVM(quantum_kernel, X_train, y_train, X_test, y_test)
# qsvm_results = qsvm.run()
# print(qsvm_results['testing_accuracy'])

In [ ]:
# Leverage Quantum Machine Learning Techniques:

# Explore quantum-enhanced neural networks and hybrid classical-quantum models.
# Variational Quantum Classifier (VQC):

In [ ]:
# from qiskit.circuit.library import TwoLocal
# from qiskit_machine_learning.algorithms.classifiers import VQC
# from qiskit.utils import QuantumInstance

# # Define variational form and quantum instance
# var_form = TwoLocal(num_qubits=3, reps=2, rotation_blocks='ry', entanglement_blocks='cz')
# quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=1024)

# # Define and train VQC
# vqc = VQC(var_form, optimizer='COBYLA', feature_map=feature_map, quantum_instance=quantum_instance)
# vqc.fit(X_train, y_train)
# vqc_accuracy = vqc.score(X_test, y_test)
# print(vqc_accuracy)

In [ ]:
# Deploy the Model:
# Create a Real-Time Prediction System:

# Develop a pipeline to process incoming climate data and generate predictions.
# Example: Flask API for real-time predictions

In [ ]:
# from flask import Flask, request, jsonify
# import joblib

# app = Flask(__name__)
# model = joblib.load('trained_model.pkl')

# @app.route('/predict', methods=['POST'])
# def predict():
#     data = request.json
#     prediction = model.predict([data['features']])
#     return jsonify({'prediction': prediction[0]})

# if __name__ == '__main__':
#     app.run(debug=True)

In [ ]:
# Develop a Visualization Dashboard:

# Use tools like Dash or Plotly to create interactive visualizations.
# Dash Dashboard for Predictions:

In [ ]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# import plotly.express as px

# app = dash.Dash(__name__)

# app.layout = html.Div([
#     dcc.Graph(id='sst-plot'),
#     dcc.Interval(id='interval-component', interval=1*1000, n_intervals=0)
# ])

# @app.callback(
#     dash.dependencies.Output('sst-plot', 'figure'),
#     [dash.dependencies.Input('interval-component', 'n_intervals')]
# )
# def update_graph(n):
#     fig = px.imshow(sst_data.mean(dim='time').values, labels=dict(color='SST Anomalies'))
#     return fig

# if __name__ == '__main__':
#     app.run_server(debug=True)

In [ ]:
# #break
# print('stop here')

In [ ]:
# #break
# print('stop here')

In [ ]:
# 4. Simulation and Analysis:

# Run quantum simulations to test the algorithm.
# Analyze results using AI techniques to identify patterns and make predictions.

In [ ]:
# # Analyze the results of the quantum circuit
# def analyze_quantum_results(counts):
#     total_shots = sum(counts.values())
#     probabilities = {state: count / total_shots for state, count in counts.items()}
#     return probabilities

# quantum_probabilities = analyze_quantum_results(quantum_results)
# print("Quantum State Probabilities:", quantum_probabilities)

In [ ]:
# 5. Model Validation:

# Compare hybrid model outcomes with traditional models.
# Validate the model using historical typhoon data.

In [ ]:
# # Validate the AI model
# historical_data_url = "https://example.com/historical_typhoon_data.csv"
# historical_data = pd.read_csv(historical_data_url)
# historical_data_normalized = (historical_data - historical_data.mean()) / historical_data.std()

# # Predict using the trained model
# historical_X = historical_data_normalized.drop('target_variable', axis=1)
# historical_y = historical_data_normalized['target_variable']
# historical_y_pred = model.predict(historical_X)

# # Evaluate the predictions
# historical_mse = mean_squared_error(historical_y, historical_y_pred)
# print("Historical Data Mean Squared Error:", historical_mse)

In [ ]:
# 6. Insight Generation:

# Generate insights on typhoon impact and propose mitigation strategies.
# Provide data-driven recommendations for policy and decision-making.
# Deliverables
# Code Repository: Public GitHub repository containing all code, data, and documentation.
# Project Report: Comprehensive summary of the project, methodology, results, and conclusions.
# Presentation Deck: 5-minute presentation summarizing the project outcomes.
# Documentation: Detailed documentation of the algorithm, data sources, and analysis.
# Next Steps
# Complete Literature Review and Data Collection.
# Develop Hybrid Algorithms with Qiskit and AI Models.
# Run Simulations and Analyze Results.
# Validate Models and Generate Insights.
# Prepare Deliverables for Submission.
# Additional Tips for Research and Development:
# Research Resources:

# Google Scholar: Google Scholar
# Semantic Scholar: Semantic Scholar
# ArXiv: ArXiv
# Q4Climate: Q4Climate
# Datasets, Models, Software:

# NCAR Data Archive: NCAR Data Archive
# NASA Earth Data: NASA Earth Data
# Data.gov: Data.gov
# Kaggle: Kaggle Datasets
# OpenDAC: OpenDAC
# Nvidia Modulus: Nvidia Modulus
# Community Earth Systems Model: CESM